<a href="https://colab.research.google.com/github/detektor777/colab_list_video/blob/main/subtitles_to_audio_elevenlabs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://elevenlabs.io/app/settings/api-keys

In [ ]:
#@title ##**Install** { display-mode: "form" }
%%capture
!pip install elevenlabs
!pip install pydub
!apt-get update && apt-get install -y ffmpeg -
!pip install python-dotenv

In [ ]:
#@title ##**Config** { display-mode: "form" }

import os
import io
from IPython.display import display, Audio, clear_output
import ipywidgets as widgets
from dotenv import load_dotenv
from elevenlabs.client import ElevenLabs
from elevenlabs import play
from pydub import AudioSegment
import tempfile

load_dotenv()

ELEVENLABS_API_KEY = "" #@param {type:"string"}
os.environ["ELEVENLABS_API_KEY"] = ELEVENLABS_API_KEY

elevenlabs = ElevenLabs(api_key=ELEVENLABS_API_KEY)

voices_data = elevenlabs.voices.get_all()
voices = dict(sorted({voice.name: voice.voice_id for voice in voices_data.voices}.items()))

additional_voices = {
    "Adam": "pNInz6obpgDQGcFmaJgB",
    "Alice": "Xb7hH8MSUJpSbSDYk0k2",
    "Antoni": "ErXwobaYiN019PkySvjV",
    "Aria": "9BWtsMINqrJLrRacOk9x",
    "Arnold": "VR6AewLTigWG4xSOukaG",
    "Arnold": "wViXBPUzp2ZZixB1xQuM",
    "Bill": "pqHfZKP75CvOlQylNhV4",
    "Brian": "nPczCjzI2devNBz1zQrb",
    "Callum": "N2lVS1w4EtoT3dr4eOWO",
    "Charlie": "IKne3meq5aSn9XLyUdCD",
    "Charlotte": "XB0fDUnXU5powFXDhCwa",
    "Chris": "iP95p4xoKVk53GoZ742B",
    "Clyde": "2EiwWnXFnvU5JabPnv8n",
    "Daniel": "onwK4e9ZLuTAKqWW03F9",
    "Dave": "CYw3kZ02Hs0563khs1Fj",
    "Domi": "AZnzlk1XvdvUeBnXmlld",
    "Dorothy": "ThT5KcBeYPX3keUQqHPh",
    "Drew": "29vD33N1CtxCmqQRPOHJ",
    "Elli": "MF3mGyEYCl7XYWbV9V6O",
    "Emily": "LcfcDJNUP1GQjkzn1xUU",
    "Eric": "cjVigY5qzO86Huf0OWal",
    "Ethan": "g5CIjZEefAph4nQFvHAz",
    "Fin": "D38z5RcWu1voky8WS1ja",
    "Freya": "jsCqWAovK2LkecY7zXl4",
    "George": "JBFqnCBsd6RMkjVDRZzb",
    "George": "Yko7PKHZNXotIFUBG7I9",
    "Gigi": "jBpfuIE2acCO8z3wKNLl",
    "Giovanni": "zcAOhNBS3c14rBihAFp1",
    "Glinda": "z9fAnlkpzviPz146aGWa",
    "Grace": "oWAxZDx7w5VEj9dCyTzz",
    "Harry": "SOYHLrjzK2X1ezoPC6cr",
    "James": "ZQe5CZNOzWyzPSCn5a3c",
    "Jeremy": "bVMeCyTHy58xNoL34h3p",
    "Jessica": "cgSgspJ2msm6clMCkdW9",
    "Jessie": "t0jbNlBVZ17f02VDIeMI",
    "Joseph": "Zlb1dXrM653N07WRdFW3",
    "Josh": "TxGEqnHWrfWFTfGW9XjX",
    "Laura": "FGY2WhTYpPnrIDTdsKH5",
    "Liam": "TX3LPaxmHKxFdv7VOQHJ",
    "Lily": "pFZP5JQG7iQjIQuC4Bku",
    "Matilda": "XrExE9yKIg1WjnnlVkGX",
    "Michael": "flq6f7yk4E4fJM5XTYuZ",
    "Mimi": "zrHiDhphv9ZnVXBqCLjz",
    "Nicole": "piTKgcLEGmPE4e6mEKli",
    "Patrick": "ODq5zmih8GrVes37Dizd",
    "Paul": "5Q0t7uMcjvnagumLfvZi",
    "Rachel": "21m00Tcm4TlvDq8ikWAM",
    "River": "SAz9YHcvj6GT2YYXdXww",
    "Roger": "CwhRBWXzGAHq8TQ4Fs17",
    "Sam": "yoZ06aMxZJJ28mfd3POQ",
    "Sarah": "EXAVITQu4vr4xnSDxMaL",
    "Serena": "pMsXgVXv3BLzUgSXRplE",
    "Thomas": "GBv7mTt0atIp3Br8iCZE",
    "Will": "bIHbv24MWmeRgasZH58o",
    "Santa Claus": "knrPHWnBmmDHMoiMeP3l"
}

for name, voice_id in additional_voices.items():
    if name not in voices:
        voices[name] = voice_id

voices = dict(sorted(voices.items()))

supported_models = {
    "Eleven Multilingual v2": "eleven_multilingual_v2",
    "Eleven Turbo v2.5": "eleven_turbo_v2_5",
    "Eleven Mono v1": "eleven_mono_v1",
    "Eleven Flash v2.5": "eleven_flash_v2_5"
}
models = supported_models

voice_dropdown = widgets.Dropdown(
    options=voices,
    description="Voice"
)

model_dropdown = widgets.Dropdown(
    options=models,
    description="Model"
)

language_dropdown = widgets.Dropdown(
    options={
        "Auto": None,
        "English": "en",
        "Russian": "ru",
        "Spanish": "es",
        "French": "fr",
        "German": "de"
    },
    description="Language"
)

stability_slider = widgets.FloatSlider(
    value=0.5,
    min=0.0,
    max=1.0,
    step=0.01,
    description="Stability",
    continuous_update=False
)

test_input = widgets.Text(
    value="Hello, world!",
    description="Test text"
)

test_button = widgets.Button(description="Test")

def save_selection(change):
    global selected_voice_id, selected_model_id, selected_language_code, stability
    selected_voice_id = voice_dropdown.value
    selected_model_id = model_dropdown.value
    selected_language_code = language_dropdown.value
    stability = stability_slider.value

def generate_audio_for_subtitle(text, voice_id, model_id, language_code=None, stability=0.5):
    try:
        voice_settings = {
            "stability": stability,
            "similarity_boost": 0.75
        }

        if model_id in ["eleven_flash_v2_5", "eleven_turbo_v2_5"] and language_code:
            audio = elevenlabs.text_to_speech.convert(
                text=text,
                voice_id=voice_id,
                model_id=model_id,
                output_format="mp3_44100_128",
                voice_settings=voice_settings,
                language_code=language_code
            )
        else:
            audio = elevenlabs.text_to_speech.convert(
                text=text,
                voice_id=voice_id,
                model_id=model_id,
                output_format="mp3_44100_128",
                voice_settings=voice_settings
            )

        audio_bytes = b""
        for chunk in audio:
            if isinstance(chunk, bytes):
                audio_bytes += chunk
        return audio_bytes
    except Exception as e:
        print(f"Ошибка генерации аудио для текста '{text}': {str(e)}")
        return None

def on_test_button_clicked(b):
    try:
        audio_bytes = generate_audio_for_subtitle(
            text=test_input.value,
            voice_id=selected_voice_id,
            model_id=selected_model_id,
            language_code=selected_language_code,
            stability=stability
        )
        if audio_bytes:
            clear_output(wait=True)
            display(voice_dropdown, model_dropdown, language_dropdown, stability_slider, test_input, test_button)
            audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")
            with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_file:
                audio_segment.export(temp_file.name, format="mp3")
                display(Audio(temp_file.name))
                os.unlink(temp_file.name)
    except Exception as e:
        print(f"Error: {str(e)}")

voice_dropdown.observe(save_selection, names="value")
model_dropdown.observe(save_selection, names="value")
language_dropdown.observe(save_selection, names="value")
stability_slider.observe(save_selection, names="value")
test_button.on_click(on_test_button_clicked)

display(voice_dropdown, model_dropdown, language_dropdown, stability_slider, test_input, test_button)

In [ ]:
#@title ##**Upload subtitles (.str)** { display-mode: "form" }
import os
import pandas as pd
import re
from IPython.display import display
from google.colab import files
from pydub import AudioSegment
import io
from datetime import datetime, timedelta
import ipywidgets as widgets

def upload_subtitle_file():
    print("Upload .srt file")
    uploaded = files.upload()
    if not uploaded:
        raise FileNotFoundError("No file uploaded")
    file_name = list(uploaded.keys())[0]
    subtitle_content = uploaded[file_name].decode('utf-8')
    return subtitle_content, file_name

def time_to_ms(time_str):
    h, m, s = time_str.replace(',', '.').split(':')
    s, ms = s.split('.')
    return int(h) * 3600000 + int(m) * 60000 + int(s) * 1000 + int(ms)

def srt_time_to_seconds(srt_time):
    hours, minutes, seconds = map(float, srt_time.replace(',', '.').split(':'))
    return hours * 3600 + minutes * 60 + seconds

def seconds_to_srt_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    secs = int(seconds % 60)
    millis = int((seconds - int(seconds)) * 1000)
    return f"{hours:02}:{minutes:02}:{secs:02},{millis:03}"

def parse_srt(srt_content):
    subtitle_blocks = re.split(r'\n\s*\n', srt_content.strip())
    subtitles = []
    for block in subtitle_blocks:
        lines = block.strip().split('\n')
        if len(lines) < 3:
            continue
        index = lines[0]
        time_line = lines[1]
        text = ' '.join(lines[2:]).strip()
        time_pattern = r'(\d{2}:\d{2}:\d{2},\d{3}) --> (\d{2}:\d{2}:\d{2},\d{3})'
        time_match = re.search(time_pattern, time_line)
        if time_match:
            start_time = time_match.group(1)
            end_time = time_match.group(2)
            duration_ms = time_to_ms(end_time) - time_to_ms(start_time)
            subtitles.append({
                'index': index,
                'start_time': start_time,
                'end_time': end_time,
                'start_ms': time_to_ms(start_time),
                'end_ms': time_to_ms(end_time),
                'duration_ms': duration_ms,
                'text': text
            })
    return subtitles

def split_subtitles_by_sentences(subtitles):
    sentences = []
    current_sentence = []
    current_start = None
    current_end = None
    for subtitle in subtitles:
        text = subtitle['text'].strip()
        start_time = srt_time_to_seconds(subtitle['start_time'])
        end_time = srt_time_to_seconds(subtitle['end_time'])
        if not text:
            continue
        if current_start is None:
            current_start = start_time
        current_end = end_time
        current_sentence.append(text)
        if text.endswith(('.', '!', '?')):
            sentences.append({
                'start': current_start,
                'end': current_end,
                'text': ' '.join(current_sentence)
            })
            current_sentence = []
            current_start = None
    if current_sentence and current_start is not None:
        sentences.append({
            'start': current_start,
            'end': current_end,
            'text': ' '.join(current_sentence)
        })
    new_subtitles = []
    for i, sentence in enumerate(sentences):
        start_time = seconds_to_srt_time(sentence['start'])
        end_time = seconds_to_srt_time(sentence['end'])
        duration_ms = (sentence['end'] - sentence['start']) * 1000
        new_subtitles.append({
            'index': str(i + 1),
            'start_time': start_time,
            'end_time': end_time,
            'start_ms': time_to_ms(start_time),
            'end_ms': time_to_ms(end_time),
            'duration_ms': duration_ms,
            'text': sentence['text']
        })
    return new_subtitles

def display_subtitles(subtitles):
    df = pd.DataFrame(subtitles)
    return df[['index', 'start_time', 'end_time', 'duration_ms', 'text']]

def display_editable_table(subtitles):
    df = display_subtitles(subtitles)
    df_styled = df.style.set_properties(**{
        'width': '100%',
        'text-align': 'left'
    }).set_table_styles([
        {'selector': 'th.col_heading', 'props': [('width', '5%'), ('text-align', 'left')]},
        {'selector': 'th.col_heading.level0.col0', 'props': [('width', '5%')]},  # index
        {'selector': 'th.col_heading.level0.col1', 'props': [('width', '15%')]},  # start_time
        {'selector': 'th.col_heading.level0.col2', 'props': [('width', '15%')]},  # end_time
        {'selector': 'th.col_heading.level0.col3', 'props': [('width', '10%')]},  # duration_ms
        {'selector': 'th.col_heading.level0.col4', 'props': [('width', '65%')]},  # text
        {'selector': 'td.col0', 'props': [('width', '5%')]},
        {'selector': 'td.col1', 'props': [('width', '15%')]},
        {'selector': 'td.col2', 'props': [('width', '15%')]},
        {'selector': 'td.col3', 'props': [('width', '10%')]},
        {'selector': 'td.col4', 'props': [('width', '65%')]}
    ])
    display(df_styled)

split_sentences_checkbox = widgets.Checkbox(
    value=False,
    description="Split by sentences",
    indent=False
)
edit_button = widgets.Button(description="Edit")

def edit_subtitles_button(b):
    global subtitles
    clear_output()
    display(split_sentences_checkbox, edit_button)
    if split_sentences_checkbox.value:
        print("Splitting subtitles by sentences...")
        subtitles = split_subtitles_by_sentences(subtitles)
    display_editable_table(subtitles)
    print(f"Showing {len(subtitles)} subtitles")

edit_button.on_click(edit_subtitles_button)

try:
    global subtitles, subtitle_filename
    subtitle_content, file_name = upload_subtitle_file()
    subtitles = parse_srt(subtitle_content)
    subtitle_filename = file_name
    if split_sentences_checkbox.value:
        print("Splitting subtitles by sentences...")
        subtitles = split_subtitles_by_sentences(subtitles)
    print(f"Processed {len(subtitles)} subtitles from {file_name}")
except Exception as e:
    print(f"Error: {str(e)}")

display(split_sentences_checkbox, edit_button)

In [ ]:
#@title ##**Str to Audio** { display-mode: "form" }
import os
import io
import tempfile
import subprocess
from pydub import AudioSegment
from IPython.display import Audio, display
from elevenlabs.client import ElevenLabs

client = ElevenLabs(api_key=ELEVENLABS_API_KEY)

supported_language_models = ["eleven_flash_v2_5", "eleven_turbo_v2_5"]

def generate_audio_for_subtitle(text, voice_id, model_id, language_code=None, stability=0.5):
    try:
        voice_settings = {
            "stability": stability,
            "similarity_boost": 0.75
        }

        if model_id in supported_language_models and language_code:
            audio = client.text_to_speech.convert(
                text=text,
                voice_id=voice_id,
                model_id=model_id,
                output_format="mp3_44100_128",
                voice_settings=voice_settings,
                language_code=language_code
            )
        else:
            audio = client.text_to_speech.convert(
                text=text,
                voice_id=voice_id,
                model_id=model_id,
                output_format="mp3_44100_128",
                voice_settings=voice_settings
            )

        audio_bytes = b""
        for chunk in audio:
            if isinstance(chunk, bytes):
                audio_bytes += chunk
        return audio_bytes
    except Exception as e:
        print(f"Error generating audio for text '{text}': {str(e)}")
        return None

def get_audio_duration(audio_bytes):
    try:
        audio = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")
        return len(audio)
    except Exception as e:
        print(f"Error measuring audio duration: {e}")
        return 0

def speed_up_audio(audio_bytes, target_duration_ms):
    try:
        with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_input:
            temp_input.write(audio_bytes)
            temp_input_path = temp_input.name

        if not os.path.exists(temp_input_path):
            raise FileNotFoundError(f"Temporary file {temp_input_path} not created")

        with tempfile.NamedTemporaryFile(suffix='.mp3', delete=False) as temp_output:
            temp_output_path = temp_output.name

        audio_duration_ms = get_audio_duration(audio_bytes)
        if audio_duration_ms <= target_duration_ms or audio_duration_ms == 0:
            return audio_bytes
        speed_factor = audio_duration_ms / target_duration_ms
        speed_factor = min(speed_factor, 2.0)

        result = subprocess.run([
            'ffmpeg',
            '-i', temp_input_path,
            '-filter:a', f'atempo={speed_factor}',
            '-vn', temp_output_path,
            '-y'
        ], check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        if result.returncode != 0:
            raise RuntimeError(f"FFmpeg error: {result.stderr.decode()}")

        with open(temp_output_path, 'rb') as f:
            sped_up_audio = f.read()

        os.unlink(temp_input_path)
        os.unlink(temp_output_path)

        return sped_up_audio

    except Exception as e:
        print(f"Error speeding up audio: {e}")
        return audio_bytes

def process_subtitles(subtitles, voice_id, model_id, language_code=None, stability=0.5):
    final_audio = AudioSegment.silent(duration=0)
    current_position_ms = 0

    total_subtitles = len(subtitles)
    print(f"Starting processing of {total_subtitles} subtitles...")

    for i, subtitle in enumerate(subtitles):
        print(f"\nProcessing subtitle {i+1}/{total_subtitles}: '{subtitle['text']}'")

        if not subtitle['text'].strip():
            silence = AudioSegment.silent(duration=subtitle['duration_ms'])
            final_audio += silence
            current_position_ms += subtitle['duration_ms']
            continue

        audio_bytes = generate_audio_for_subtitle(
            subtitle['text'], voice_id, model_id, language_code, stability
        )
        if audio_bytes is None:
            continue

        audio_duration_ms = get_audio_duration(audio_bytes)

        min_start_ms = 0 if i == 0 else subtitles[i-1]['end_ms']
        ideal_start_ms = subtitle['start_ms']
        max_end_ms = subtitles[i+1]['start_ms'] if i < total_subtitles - 1 else subtitle['end_ms'] + 1000

        start_ms = ideal_start_ms
        step = 1000
        while start_ms >= min_start_ms:
            if start_ms + audio_duration_ms <= max_end_ms:
                break
            start_ms -= step
        else:
            start_ms = min_start_ms
            if start_ms + audio_duration_ms > max_end_ms:
                available_time = max_end_ms - start_ms
                print(f"Speeding up audio to {available_time}ms")
                audio_bytes = speed_up_audio(audio_bytes, available_time)
                audio_duration_ms = get_audio_duration(audio_bytes)

        silence_before = start_ms - current_position_ms
        if silence_before > 0:
            final_audio += AudioSegment.silent(duration=silence_before)
            current_position_ms += silence_before

        audio_segment = AudioSegment.from_file(io.BytesIO(audio_bytes), format="mp3")
        final_audio += audio_segment
        current_position_ms += audio_duration_ms

        if i < total_subtitles - 1 and current_position_ms < subtitles[i+1]['start_ms']:
            silence_after = subtitles[i+1]['start_ms'] - current_position_ms
            final_audio += AudioSegment.silent(duration=silence_after)
            current_position_ms += silence_after

    output_filename = "subtitle_audio.mp3"
    final_audio.export(output_filename, format="mp3")
    print(f"Audio saved: {output_filename}")
    display(Audio(output_filename))

!apt-get install -y ffmpeg

if 'subtitles' in globals() and 'selected_voice_id' in globals() and 'selected_model_id' in globals():
    print(f"Processing {len(subtitles)} subtitles with voice {selected_voice_id}")
    process_subtitles(
        subtitles,
        selected_voice_id,
        selected_model_id,
        selected_language_code,
        stability
    )
else:
    print("Subtitles or voice parameters not loaded. Please run previous cells.")